
# Beat the Machine: Optimization Tutorial

We time each approach and count inner-loop work to understand **where the compute cost lives**
and how **early pruning + loop ordering** speeds things up.

**Problem:** Use digits 1–9 exactly once to form three 3-digit numbers ABC + DEF = GHI with:
1. A < D < G
2. C even, F odd
3. (G + H + I) % 9 == 0


## V0 — Naive nested loops (checks late)

In [1]:

from collections import defaultdict

digits = list(range(1,10))

def as_abc(a,b,c):
    return 100*a + 10*b + c

def solve_v0_naive():
    counters = defaultdict(int)
    solutions = []

    for A in digits:
        for B in [x for x in digits if x != A]:
            for C in [x for x in digits if x not in (A,B)]:
                for D in [x for x in digits if x not in (A,B,C)]:
                    for E in [x for x in digits if x not in (A,B,C,D)]:
                        for F in [x for x in digits if x not in (A,B,C,D,E)]:
                            for G in [x for x in digits if x not in (A,B,C,D,E,F)]:
                                for H in [x for x in digits if x not in (A,B,C,D,E,F,G)]:
                                    for I in [x for x in digits if x not in (A,B,C,D,E,F,G,H)]:
                                        counters["v0_innermost"] += 1
                                        if not (A < D < G):
                                            continue
                                        if C % 2 != 0 or F % 2 != 1:
                                            continue
                                        if (G + H + I) % 9 != 0:
                                            continue
                                        ABC = as_abc(A,B,C)
                                        DEF = as_abc(D,E,F)
                                        GHI = as_abc(G,H,I)
                                        if ABC + DEF == GHI:
                                            solutions.append((ABC, DEF, GHI))
    return solutions, counters

sols, counters = solve_v0_naive()
len(sols), sorted(sols)[:3], counters

(136,287,459)


(136, 287, 459)

## V1 — Permutations baseline

In [2]:

from itertools import permutations
from collections import defaultdict

digits = list(range(1,10))

def as_abc(a,b,c):
    return 100*a + 10*b + c

def solve_v1_permutations():
    counters = defaultdict(int)
    solutions = []
    for A,B,C,D,E,F,G,H,I in permutations(digits, 9):
        counters["v1_innermost"] += 1
        if not (A < D < G):
            continue
        if C % 2 != 0 or F % 2 != 1:
            continue
        if (G + H + I) % 9 != 0:
            continue
        ABC = as_abc(A,B,C)
        DEF = as_abc(D,E,F)
        GHI = as_abc(G,H,I)
        if ABC + DEF == GHI:
            solutions.append((ABC, DEF, GHI))
    return solutions, counters

sols, counters = solve_v1_permutations()
len(sols), sorted(sols)[:3], counters


(52,
 [(124, 659, 783), (128, 367, 495), (128, 439, 567)],
 defaultdict(int, {'v1_innermost': 362880}))

## V2 — Early pruning at each depth

In [3]:

from collections import defaultdict

digits = list(range(1,10))

def as_abc(a,b,c):
    return 100*a + 10*b + c

def solve_v2_prune_early():
    counters = defaultdict(int)
    solutions = []

    for A in digits:
        for D in [x for x in digits if x != A and x > A]:
            for G in [x for x in digits if x not in (A, D) and x > D]:
                used_ADG = {A, D, G}

                for C in [x for x in digits if x not in used_ADG and x % 2 == 0]:
                    for F in [x for x in digits if x not in used_ADG and x != C and x % 2 == 1]:
                        used_ADGCF = used_ADG | {C, F}

                        for B in [x for x in digits if x not in used_ADGCF]:
                            for E in [x for x in digits if x not in used_ADGCF and x != B]:
                                used_up_to_E = used_ADGCF | {B, E}
                                for H in [x for x in digits if x not in used_up_to_E]:
                                    for I in [x for x in digits if x not in used_up_to_E and x != H]:
                                        counters["v2_innermost"] += 1
                                        if (G + H + I) % 9 != 0:
                                            continue
                                        ABC = as_abc(A,B,C)
                                        DEF = as_abc(D,E,F)
                                        GHI = as_abc(G,H,I)
                                        if ABC + DEF == GHI:
                                            solutions.append((ABC, DEF, GHI))
    return solutions, counters

sols, counters = solve_v2_prune_early()
len(sols), sorted(sols)[:3], counters


(52,
 [(124, 659, 783), (128, 367, 495), (128, 439, 567)],
 defaultdict(int, {'v2_innermost': 16800}))

## V3 — Order + carry pruning + early divisibility

In [4]:

from collections import defaultdict

digits = list(range(1,10))

def as_abc(a,b,c):
    return 100*a + 10*b + c

def solve_v3_carry_and_ordering():
    counters = defaultdict(int)
    solutions = []

    for G in digits:
        for H in [x for x in digits if x != G]:
            for I in [x for x in digits if x not in (G,H)]:
                counters["v3_pick_GHI"] += 1
                if (G + H + I) % 9 != 0:
                    continue
                used_GHI = {G,H,I}

                lesser = [x for x in digits if x not in used_GHI and x < G]
                for A in lesser:
                    for D in [x for x in lesser if x > A]:
                        used_ADGHI = used_GHI | {A, D}

                        for C in [x for x in digits if x not in used_ADGHI and x % 2 == 0]:
                            for F in [x for x in digits if x not in used_ADGHI and x != C and x % 2 == 1]:
                                counters["v3_units_pairs"] += 1

                                u = C + F
                                if I != (u % 10):
                                    continue
                                carry_u = u // 10

                                used_ADGHI_CF = used_ADGHI | {C, F}

                                tens_candidates = [ (B,E) for B in digits if B not in used_ADGHI_CF
                                                   for E in digits if E not in used_ADGHI_CF and E != B ]
                                counters["v3_tens_pairs_considered"] += len(tens_candidates)

                                for B,E in tens_candidates:
                                    counters["v3_tens_pairs_loop"] += 1
                                    t = B + E + carry_u
                                    if H != (t % 10):
                                        continue
                                    carry_t = t // 10

                                    h = A + D + carry_t
                                    if G != (h % 10):
                                        continue
                                    carry_h = h // 10
                                    if carry_h != 0:
                                        continue

                                    ABC = as_abc(A,B,C)
                                    DEF = as_abc(D,E,F)
                                    GHI = as_abc(G,H,I)
                                    if ABC + DEF == GHI:
                                        solutions.append((ABC, DEF, GHI))

    solutions = sorted(set(solutions))
    return solutions, counters

sols, counters = solve_v3_carry_and_ordering()
len(sols), sorted(sols)[:3], counters


(52,
 [(124, 659, 783), (128, 367, 495), (128, 439, 567)],
 defaultdict(int,
             {'v3_pick_GHI': 504,
              'v3_units_pairs': 1080,
              'v3_tens_pairs_considered': 312,
              'v3_tens_pairs_loop': 312}))

## Timing comparison

In [5]:

import time
import pandas as pd

def run_and_time(func, name):
    t0 = time.perf_counter()
    sols, counters = func()
    t1 = time.perf_counter()
    return {"Version": name, "Seconds": t1 - t0, "Solutions": len(sols), **counters}

rows = []
for name, fn in [
    ("V0: Naive nested loops (late checks)", solve_v0_naive),
    ("V1: Permutations baseline",            solve_v1_permutations),
    ("V2: Early pruning at each depth",      solve_v2_prune_early),
    ("V3: Order + carry pruning + early div",solve_v3_carry_and_ordering),
]:
    rows.append(run_and_time(fn, name))

pd.DataFrame(rows).fillna(0)


,Version,Seconds,Solutions,v0_innermost,v1_innermost,v2_innermost,v3_pick_GHI,v3_units_pairs,v3_tens_pairs_considered,v3_tens_pairs_loop
0,V0: Naive nested loops (late checks),0.975911,52,362880.0,0.0,0.0,0.0,0.0,0.0,0.0
1,V1: Permutations baseline,0.053786,52,0.0,362880.0,0.0,0.0,0.0,0.0,0.0
2,V2: Early pruning at each depth,0.021521,52,0.0,0.0,16800.0,0.0,0.0,0.0,0.0
3,V3: Order + carry pruning + early div,0.001480,52,0.0,0.0,0.0,504.0,1080.0,312.0,312.0
